In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras')

In [ ]:
from utils import generate_generator, gen_imgPaths_and_labelPaths, monitor_mae, monitor_mse,relative_headcount_loss,customloss

In [ ]:
#eatch patch shape
import cv2
from google.colab.patches import cv2_imshow

a=cv2.imread('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/test_data/images/1.jpg')
a.shape
#cv2_imshow(a)

In [ ]:
import os
img_path = '/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/test_data/images/'
test_paths = sorted([img_path + p for p in os.listdir(img_path)], key=lambda x: int(x[:-len('.jpg')].split('/')[-1]))

train_path = '/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/formatted_trainval/Shanghaitech_patches_9/train/'
train_paths = sorted([train_path + p for p in os.listdir(train_path)], key=lambda x: float(x[:-len('.jpg')].split('/')[-1].replace('_', '.')))
print(train_paths[0])
print(test_paths[0])

In [ ]:
#density maps shapes

import pandas as pd
df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/formatted_trainval/Shanghaitech_patches_9/train_den/2_1.csv")
df.shape

In [ ]:
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Conv2D, MaxPooling2D, Concatenate, Input
from keras.models import load_model, load_model, Model
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
from Sacnn import SACNN
from utils import generate_generator, gen_imgPaths_and_labelPaths, monitor_mae, monitor_mse,relative_headcount_loss,customloss


# Settings
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
%matplotlib inline

In [ ]:
# Data Import
train_paths, train_labels, validation_paths, validation_labels, test_paths, test_labels = gen_imgPaths_and_labelPaths()
for i in ['train_paths', 'train_labels', 'validation_paths', 'validation_labels', 'test_paths', 'test_labels']:
    print('len({}) ='.format(i), len(eval(i)))

In [ ]:
# #  Data Generation and caching the files as .npy
batch_size_train = 1
epochs_per_iteration = 1
batch_size_validation = 9  # len(validation_labels) // 4
batch_size_test = 1
train_generator = generate_generator(train_paths, train_labels, batch_size=len(train_paths))
validation_generator = generate_generator(validation_paths, validation_labels, batch_size=len(validation_paths))
test_generator = generate_generator(test_paths, test_labels, batch_size=len(test_paths))

train_img, train_labels = train_generator.__next__()
print('Train data reading is done.')
validation_img, validation_labels = validation_generator.__next__()
print('Validation data reading is done.')
test_img, test_labels = test_generator.__next__()
print('Test data reading is done.')

np.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/train_img.npy', train_img)
np.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/train_labels.npy', train_labels)
np.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/validation_img.npy', validation_img)
np.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/validation_labels.npy', validation_labels)
np.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/test_img.npy', test_img)
np.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/test_labels.npy', test_labels)

In [ ]:
np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/test_img1.npy')

In [ ]:
# Read all data into memory   
train_img, train_labels, validation_img, validation_labels, test_img, test_labels = (
    np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/train_img.npy'), np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/train_labels.npy'), np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/validation_img.npy'),
    np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/validation_labels.npy'), np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/test_img.npy'), np.load('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/DataPreparation/test_labels.npy')
)

In [ ]:
print(train_img.shape,train_labels.shape,test_img.shape,test_labels.shape,validation_img.shape,validation_labels.shape)

In [ ]:
# Training
weights_dir = '/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights'
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)

checkpointer_best_val = ModelCheckpoint(
    filepath=os.path.join(weights_dir, 'Sacnn_val_monitoradam3.hdf5'),
    monitor='val_loss', verbose=1, save_best_only=True, mode='min'
)
checkpointer_best_train = ModelCheckpoint(
    filepath=os.path.join(weights_dir, 'Sacnn_train_monitoradam3.hdf5'),
    monitor='loss', verbose=1, save_best_only=True, mode='min'
)
model = SACNN()
model.compile(loss=monitor_mse, optimizer=Adam(lr=1e-4), metrics=[monitor_mae, monitor_mse])

# Loading the model
model = load_model('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn_val.hdf5', custom_objects={'monitor_mae': monitor_mae, 'monitor_mse': monitor_mse})
print(model)
print('Training ...')
# history = model.fit_generator(
#     train_generator, steps_per_epoch=np.ceil(len(train_paths)/batch_size_train/epochs_per_iteration),
#     epochs=epochs_per_iteration*200,
#     validation_data=validation_generator, validation_steps=len(validation_labels)/batch_size_validation//2,
#     verbose=1, callbacks=[checkpointer_best]
# )
history = model.fit(
    train_img, train_labels, batch_size=1, epochs=3,validation_data=(validation_img, validation_labels),
     callbacks=[checkpointer_best_val, checkpointer_best_train]

)


model.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn_moniadam3.hdf5')
model.save_weights('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn_weightsadam3.hdf5',overwrite=True)

In [ ]:
# Saving model weights
model1=load_model('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn50.hdf5', custom_objects={'monitor_mae': monitor_mae, 'monitor_mse': monitor_mse})
model1.compile(loss=customloss, optimizer=Adam(lr=1e-3), metrics=[monitor_mae, monitor_mse])
history = model1.fit(
    train_img, train_labels, batch_size=1, epochs=100,validation_data=(validation_img, validation_labels),
     callbacks=[checkpointer_best_val, checkpointer_best_train]
)

model1.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn200.hdf5')
model1.save_weights('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn_weights200.hdf5',overwrite=True)


In [ ]:
#Visuvalizing Filters
# summarize filter shapes
for layer in model.layers:
	# check for convolutional layer
	if 'conv' not in layer.name:
		continue
	# get filter weights
	filters, biases = layer.get_weights()
	print(layer.name, filters.shape)

In [ ]:
filters, biases = model.layers[1].get_weights()
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
#print(filters)
n_filters, ix = 6, 1
for i in range(n_filters):
	# get the filter
  f = filters[:, :, :, i]
  print(f)
	# plot each channel separately
  for j in range(3):
		# specify subplot and turn of axis
    ax = plt.subplot(n_filters, 3, ix)
    ax.set_xticks([])
    ax.set_yticks([])
    # plot filter channel in grayscale
    plt.imshow(f[:, :, j], cmap='gray')
    ix += 1
# show the figure
plt.show()

In [ ]:
val_loss, loss = history.history['val_loss'], history.history['loss']
loss = np.asarray(loss)
# loss[loss > 0.0002] = 0.0002
plt.plot(loss, 'b')
plt.legend(['loss'])
plt.show()
plt.plot(val_loss, 'r')
plt.legend(['val_loss'])
plt.show()

In [ ]:
print(np.squeeze(test_img[0]))
st=np.squeeze(test_labels[0])
count=np.sum(st)
print(count)

In [ ]:
m1=load_model('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn_200.hdf5',custom_objects={'monitor_mae': monitor_mae, 'monitor_mse': monitor_mse})
m1.summary()

In [ ]:
m1.compile(loss=relative_headcount_loss, optimizer=SGD(lr=1e-4), metrics=[monitor_mae, monitor_mse])
print("Training.....")
history = m1.fit(
    train_img, train_labels, batch_size=1, epochs=10,validation_data=(validation_img, validation_labels),
)
m1.save('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn210withReloss.hdf5')
m1.save_weights('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/weights/Sacnn210_weights_withReloss.hdf5',overwrite=True)

In [ ]:
val_loss, loss = history.history['val_loss'], history.history['loss']
loss = np.asarray(loss)
# loss[loss > 0.0002] = 0.0002
plt.plot(loss, 'b')
plt.legend(['loss'])
plt.show()
plt.plot(val_loss, 'r')
plt.legend(['val_loss'])
plt.show()

In [ ]:
# Testing a single sample
import cv2
from google.colab.patches import cv2_imshow

a=cv2.imread('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/test_data/images/1.jpg')
#a=cv2.imread('/content/gdrive/My Drive/Colab Notebooks/SacnnKeras/datasets/ShanghaiTech/test_data/ground_truth/5.jpg')
cv2_imshow(a)
#print(a.shape)
gray= cv2.cvtColor(a, cv2.COLOR_RGB2BGR)
#cv2_imshow(gray)
#print(gray.shape)
#a=np.expand_dims(gray,3)
#a=np.expand_dims(a,0)
#print(a.shape)
ips= np.reshape(gray, [1, *gray.shape[:2], 1])
#predict
op=np.squeeze(m1.predict(ips))


np.sum(op)

In [ ]:
fg, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16, 5))
ax0.imshow(np.squeeze(ips), cmap='gray')
ax2.imshow(op * (255 / (np.max(op) - np.min(op))), cmap='color')
plt.show()

In [ ]:
# Testing with a #num_test sample size
model = load_model('../Multi_column_CNN/weights/mcnn_B.hdf5', custom_objects={'monitor_mae': monitor_mae, 'monitor_mse': monitor_mse})
ae = []
se = []
num_test = 20
for i in range(test_img.shape[0])[:num_test]:
    inputs = np.reshape(test_img[i], [1, *test_img[i].shape[:2], 1])
    outputs = np.squeeze(model.predict(inputs))
#     outputs[outputs < np.mean(outputs)] = 0
    density_map = np.squeeze(test_labels[i])
    count_label = np.sum(density_map)
    count_prediction = np.sum(outputs)
    fg, (ax0, ax1, ax2) = plt.subplots(1, 3, figsize=(16, 5))
    plt.suptitle(' '.join([
        'count_label:', str(round(count_label, 3)),
        'count_prediction:', str(round(count_prediction, 3))
    ]))
    ax0.imshow(np.squeeze(inputs), cmap='gray')
    ax1.imshow(density_map * (255 / (np.max(density_map) - np.min(density_map))), cmap='gray')
    ax2.imshow(outputs * (255 / (np.max(outputs) - np.min(outputs))), cmap='gray')
    plt.show()
    ae.append(abs(count_label - count_prediction))
    se.append((count_label - count_prediction) ** 2)
mae = np.mean(ae)
mse = np.mean(se)
print('mae:', mae, 'mse:', mse)